# Mean Average Precision Report

This notebooks provides an overview for using and understanding the mean average precision report check.

**Structure:**

- [What is the purpose of the check?](#purpose)
- [Generate data & model](#generate)
- [Defining the Prediction Formatter](#pred_format)
- [Run the check](#run_check)
- [Define a condition](#define_condition)

## What is the purpose of the check? <a name='purpose'></a>

The mean average precision report evaluates the mean average precision metric on the given model and data. 
The check only works on object detection as it computes the value with the IoU and Area size.

### Imports

In [1]:
import numpy as np

from deepchecks.vision.base import VisionData
from deepchecks.vision.checks.performance import MeanAveragePrecisionReport
from deepchecks.vision.datasets.classification import mnist

### Generate Data and Model <a name='generate'></a>

We generate a sample dataset of 128 images from the [COCO dataset](https://cocodataset.org/#home), and using the [YOLOv5 model](https://github.com/ultralytics/yolov5).

For the label formatter - our dataset returns exactly the accepted format, so our formatting function is the simple `lambda x: x` function.

In [2]:
yolo = mnist.load_model(pretrained=True)

test_ds = mnist.load_dataset(train=False, object_type='VisionData')

In [3]:
test_ds

In [4]:
from deepchecks.vision import Suite

### Run the check <a name='run_check'></a>

In [5]:
check = Suite('ahh', MeanAveragePrecisionReport())
result = check.run(test_ds, model=yolo)
result

ahh - Train:   0%|          | 0/10 [00:00<?, ? Check/s]

mahhh


### Observe the check’s output

The result value is a dataframe that has the average precision score per each area range and IoU.

In [4]:
result.value

,mAP@0.5..0.95 (%),AP@.50 (%),AP@.75 (%)
Area size,,,
All,0.409436,0.566673,0.425339
Small (area < 32^2),0.212816,0.342429,0.212868
Medium (32^2 < area < 96^2),0.383089,0.600228,0.349863
Large (area < 96^2),0.541146,0.674493,0.585378


## Define a condition <a name='define_condition'></a>

We can define a condition that enforce our model's average precision score is not less than a given threshold

In [5]:
check = MeanAveragePrecisionReport().add_condition_test_average_precision_not_less_than(0.4)
result = check.run(test_ds, yolo)
result.show(show_additional_outputs=False)

Status,Condition,More Info
✖,Scores are not less than 0.4,"Found scores below threshold: {'Small (area < 32^2)': {'AP@.50 (%)': '0.342'}, 'Medium (32^2 < area < 96^2)': {'AP@.75 (%)': '0.35'}}"
